In [1]:
# facadesデータセットのダウンロード
# cityscapes, night2day, edges2handbags, edges2shoes, facades, mapsも同様
!wget https://cmp.felk.cvut.cz/~tylecr1/facade/CMP_facade_DB_base.zip
!mkdir -p ./data/facades
!unzip CMP_facade_DB_base.zip -d ./data/facades/
!rm CMP_facade_DB_base.zip

#!wget http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/facades.tar.gz
#!mkdir -p ./datasets/facades
#!tar -zxvf facades.tar.gz -C ./datasets/
#!rm facades.tar.gz


--2022-04-08 18:17:20--  https://cmp.felk.cvut.cz/~tylecr1/facade/CMP_facade_DB_base.zip
cmp.felk.cvut.cz (cmp.felk.cvut.cz) をDNSに問いあわせています... 147.32.84.2
cmp.felk.cvut.cz (cmp.felk.cvut.cz)|147.32.84.2|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 36517004 (35M) [application/zip]
`CMP_facade_DB_base.zip' に保存中

CMP_facade_DB_base. 100%[===================>]  34.83M  4.76MB/s    in 8.6s    

2022-04-08 18:17:30 (4.04 MB/s) - `CMP_facade_DB_base.zip' へ保存完了 [36517004/36517004]

Archive:  CMP_facade_DB_base.zip
   creating: ./data/facades/base/
  inflating: ./data/facades/base/cmp_b0001.jpg  
  inflating: ./data/facades/base/cmp_b0001.png  
  inflating: ./data/facades/base/cmp_b0001.xml  
  inflating: ./data/facades/base/cmp_b0002.jpg  
  inflating: ./data/facades/base/cmp_b0002.png  
  inflating: ./data/facades/base/cmp_b0002.xml  
  inflating: ./data/facades/base/cmp_b0003.jpg  
  inflating: ./data/facades/base/cmp_b0003.png  
  inflating: ./data/facades/base/cmp_b

In [4]:
!pip install torch>=0.4.1
!pip install torchvision>=0.2.1
!pip install dominate>=2.3.1
!pip install visdom>=0.1.8.3

ERROR: Could not install packages due to an OSError: [Errno 30] 読み込み専用ファイルシステムです: '/home/sugiura/.local/lib'

ERROR: Could not install packages due to an OSError: [Errno 30] 読み込み専用ファイルシステムです: '/home/sugiura/.local/lib'

ERROR: Could not install packages due to an OSError: [Errno 30] 読み込み専用ファイルシステムです: '/home/sugiura/.local/lib'

ERROR: Could not install packages due to an OSError: [Errno 30] 読み込み専用ファイルシステムです: '/home/sugiura/.local/lib'



In [6]:
import sys
import argparse
import os.path
import random
import time
import datetime
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.utils as vutils
from PIL import Image
# from visdom import Visdom
import pickle

In [7]:
# 条件画像と正解画像のペアデータセット生成クラス
class AlignedDataset(Dataset):
    # configは全ての学習条件を格納する

    # 画像データは'/path/to/data/train'および'/path/to/data/test'に
    # {A,B}の形式で格納されているものとみなす

    def __init__(self, config):
        # データセットクラスの初期化
        self.config = config

        # データディレクトリの取得

        #パス名操作に関する処理をまとめたモジュールに実装されている関数の一つです.
        #引数に与えられた二つの文字列を結合させ、一つのパスにする事ができます.
        #二つの間には / が自動で入る
        dir = os.path.join(config.dataroot, config.phase)
        # 画像データパスの取得

        #組み込み関数sorted(): ソートした新たなリストを生成
        #昇順
        #正解画像
        self.A_paths = sorted(self.__make_dataset_A(dir))
        #ラベル画像
        self.B_paths = sorted(self.__make_dataset_B(dir))

    @classmethod
    def is_image_file_png(self, fname):
        # 画像ファイルかどうかを返す
        #fnameに入っているものの末尾がext(ここではpngかjpg)なのかどうかを判断している．
        #一致しているなら，trueを返す．
        return fname.endswith('.png')
    @classmethod
    def is_image_file_jpg(self, fname):
        # 画像ファイルかどうかを返す
        #fnameに入っているものの末尾がext(ここではpngかjpg)なのかどうかを判断している．
        #一致しているなら，trueを返す．
        return fname.endswith('jpg')

    @classmethod
    #正解画像データセット作成
    def __make_dataset_A(self, dir):
        # 画像データセットをメモリに格納
        # 画像データセットをメモリに格納
        images_real = []

        #ディレクトリの確認
        assert os.path.isdir(dir), '%s is not a valid directory' % dir

        #root:現在のディレクトリ
        # _:内包するディレクトリ
        #fnames:内包するファイル
        for root, _, fnames in sorted(os.walk(dir)):
            for fname in fnames:
                if self.is_image_file_jpg(fname):
                    path = os.path.join(root, fname)
                    images_real.append(path)

        return images_real

    #ラベル画像データセット作成
    @classmethod
    def __make_dataset_B(self, dir):

        images_label = []

        #ディレクトリの確認
        assert os.path.isdir(dir), '%s is not a valid directory' % dir

        #root:現在のディレクトリ
        # _:内包するディレクトリ
        #fnames:内包するファイル
        for root, _, fnames in sorted(os.walk(dir)):
            for fname in fnames:
                if self.is_image_file_png(fname):
                    path = os.path.join(root, fname)
                    images_label.append(path)
        
        return images_label

    def __getitem__(self, index):
        # 学習用データ１つの生成
        # A(テンソル) : 条件画像
        # B(テンソル) : Aのペアとなるターゲット画像

        # ランダムなindexの画像を取得 
        B_path = self.B_paths[index]
        #Image.open(a):aで指定した画像を開く
        #RGB画像に変換
        
        B = Image.open(B_path).convert('RGB')

        load_size = self.config.load_size
        #transform = transforms.Resize([load_size, load_size], Image.BICUBIC)
        B = B.resize((load_size, load_size), Image.NEAREST)
        B = np.array(B)

        #H*W*CをC*H*Wに．指定の仕方は，元の位置をどこに持っていくかになる．
        #Hについていうと，Hはもともと0番目にある．その0を持ってくるところに書く

        B = B.transpose(2, 0, 1)
        label_image = torch.from_numpy(B)

        return label_image

    def __len__(self):
        # 全画像ファイル数を返す
        return len(self.A_paths)

In [8]:
class Opts():
    def __init__(self):
        self.epochs = 100
        self.save_data_interval = 10
        self.save_image_interval = 10
        self.log_interval = 20
        self.sample_interval = 10
        self.batch_size = 64
        self.load_size = 286
        self.crop_size = 256
        self.cpu = True
        self.dataroot = 'data/facades'
        self.output_dir = 'output'
        self.log_dir = './logs'
        self.phase = 'base'
        self.lambda_L1 = 100.0
        self.epochs_lr_decay = 0
        self.epochs_lr_decay_start = -1
        self.path_to_generator = None
        self.path_to_discriminator = None
        self.device_name = "cuda:0"
        self.device = torch.device(self.device_name)
    
    def to_dict(self):
        parameters = {
            'epochs': self.epochs,
            'save_data_interval': self.save_data_interval,
            'save_image_interval': self.save_image_interval,
            'log_interval': self.log_interval,
            'sample_interval': self.sample_interval,
            'batch_size': self.batch_size,
            'load_size': self.load_size,
            'crop_size': self.crop_size,
            'cpu': self.cpu,
            'dataroot': self.dataroot,
            'output_dir': self.output_dir,
            'log_dir': self.log_dir,
            'phase': self.phase,
            'lambda_L1': self.lambda_L1,
            'epochs_lr_decay': self.epochs_lr_decay,
            'epochs_lr_decay_start': self.epochs_lr_decay_start,
            'path_to_generator': self.path_to_generator,
            'path_to_discriminator': self.path_to_discriminator,
            'device_name': self.device_name,
        }
        return parameters

opt = Opts()

param_save_path = os.path.join('output', 'param.json')

In [9]:
#RGB値とマッチしたときのインデックスを返す
def label_RGB_value(RGB_value):
    default_RGB_value = [
                        [255,85,0],   
                        [255,170,0],   
                        [0,85,255],   
                        [0,0,255],   
                        [0,0,170],   
                        [170,255,85],   
                        [170,0,0],   
                        [0,170,255],   
                        [255,255,0],   
                        [85,255,170],   
                        [255,0,0],   
                        [0,255,255],   
                        ]
    default_RGB_value = np.array(default_RGB_value)

    #print(RGB_value)
    label_index = 0
    for i in range(12):
      if np.array_equal(default_RGB_value[i],RGB_value):
        label_index = i

    #print(label_index)

    return label_index

In [10]:
#任意の位置のRGBの値を取得し，リストに追加
def get_pixel_RGB_value(H,W,label_image_numpy):
  RGB_value = np.zeros(3)
  
  for i in range(3):
    RGB_value[i] = label_image_numpy[i, H, W]

  return RGB_value

In [11]:
def cal_multichannel(label_image_numpy):
  #縦，横のサイズを指定
  crop_size = 256

  #0で埋められた二次元ndarrayを用意．大きさは縦*横
  z = torch.zeros(12, crop_size, crop_size)

  for h in range(crop_size):
    for w in range(crop_size):
      RGB_value = get_pixel_RGB_value(h, w, label_image_numpy)
      label = label_RGB_value(RGB_value)

      if label == 0:
        z[0, h, w] = 1
      elif label == 1:  
        z[1, h, w] = 1
      elif label == 2: 
        z[2, h, w] = 1
      elif label == 3:
        z[3, h, w] = 1
      elif label == 4: 
        z[4, h, w] = 1
      elif label == 5: 
        z[5, h, w] = 1
      elif label == 6: 
        z[6, h, w] = 1
      elif label == 7: 
        z[7, h, w] = 1
      elif label == 8: 
        z[8, h, w] = 1
      elif label == 9: 
        z[9, h, w] = 1
      elif label == 10: 
        z[10,h, w] = 1
      elif label == 11:
        z[11, h, w] = 1

  return z

In [12]:
dataset = AlignedDataset(opt)
i = 0
img_set = []
for label_image_tensor in dataset:
    label_image_numpy = label_image_tensor.numpy()
    oneHot = cal_multichannel(label_image_numpy)
    img = oneHot.to('cpu').detach().numpy()
    img_set.append(img)

img_forth_dim = np.stack(img_set)
print(img_forth_dim.shape)
  
  # for i in range(12):
  #   gray_scale_img = np.zeros((256,256))
  #   for j in range(256):
  #     for k in range(256):
  #       gray_scale_img[j,k] = img[i,j,k]
    
with open('img_numpy.pickle', mode='wb') as f:
  pickle.dump(img_forth_dim, f)



(378, 12, 256, 256)
